In [66]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import streamlit as st
from pycaret.regression import *

In [67]:
### preprocessing 

def split_category(value):
    vals = []
    if '|' in value:
        vals = value.split('|')
    else:
        vals.append(value)
    return vals

def split_columns_with_multiple_values(data):
    data['Airline'] = data['Airline'].apply(split_category)
    data['Places Covered'] = data['Places Covered'].apply(split_category)
    data['Sightseeing Places Covered'] = data['Sightseeing Places Covered'].apply(split_category)
    data['Hotel Details'] = data['Hotel Details'].apply(split_category)
    return data

def one_hot_encoding_on_columns(data):
    dummy_type = pd.get_dummies(data['Package Type'], prefix='type')
    data.drop(columns=['Package Type'], inplace=True)
    data = pd.concat([data, dummy_type], axis=1)

    dummy_city = pd.get_dummies(data['Start City'], prefix='sc')
    data.drop(columns=['Start City'], inplace=True)
    data = pd.concat([data, dummy_city], axis=1)
    return data

def show_category(series):
    values = {}
    for val in series:
        for each in val:
            if each in values:
                values[each] += 1
            else:
                values[each] = 1
    return values

def make_feature_col(series, all_keys):
    feature_dict = {}
    for key in all_keys:
        feature_dict[key] = []

    for items in series:
        for key in all_keys:
            if key not in items:
                feature_dict[key].append(0)
            else:
                feature_dict[key].append(1)

    return pd.DataFrame(feature_dict)

def structuring_columns(data):
    change = lambda pc: [each+'_ae' for each in pc]
    A_all_keys = show_category(data['Airline']).keys()
    A_all_keys = change(A_all_keys)
    airline = make_feature_col(data['Airline'], A_all_keys)
    data = pd.concat([data, airline], axis=1)
    
    change = lambda pc: [each+'_pc' for each in pc]
    PC_all_keys = show_category(data['Places Covered']).keys()
    PC_all_keys = change(PC_all_keys)
    p_covered = make_feature_col(data['Places Covered'], PC_all_keys)
    data = pd.concat([data, p_covered], axis=1)

    change = lambda pc: [each+'_hd' for each in pc]
    HD_all_keys = show_category(data['Hotel Details']).keys()
    HD_all_keys = change(PC_all_keys)
    h_details = make_feature_col(data['Hotel Details'], HD_all_keys)
    data = pd.concat([data, h_details], axis=1)

    return data

def remove_useless_columns(data):
    data.drop('Uniq Id', inplace=True, axis=1)
    data.drop('Package Name', inplace=True, axis=1)
    data.drop('Destination', inplace=True, axis=1)
    data.drop('Cancellation Rules', inplace=True, axis=1)
    return data

def hotel_details_encoding(data):
    data['Hotel Details'] = data['Hotel Details'].str.replace(":One",':1.0')
    data['Hotel Details'] = data['Hotel Details'].str.replace(":Two",':2.0')
    data['Hotel Details'] = data['Hotel Details'].str.replace(":Three",':3.0')
    data['Hotel Details'] = data['Hotel Details'].str.replace(":Four",':4.0')
    data['Hotel Details'] = data['Hotel Details'].str.replace(":Five",':5.0')

    data['Hotel Details'] = data['Hotel Details'].str.replace(" - mmt holidays special",'')
    data['Hotel Details'] = data['Hotel Details'].str.replace("- mmt holidays special",'')
    data['Hotel Details'] = data['Hotel Details'].str.replace("-mmt holidays special",'')
    data['Hotel Details'] = data['Hotel Details'].str.replace("(mmt holidays special)",'')
    data['Hotel Details'] = data['Hotel Details'].str.replace(" - mmt special",'')
    data['Hotel Details'] = data['Hotel Details'].str.replace("- mmt speciall",'')
    data['Hotel Details'] = data['Hotel Details'].str.replace("-mmt special",'')
    data['Hotel Details'] = data['Hotel Details'].str.replace("mmt special",'')
    return data

def convert_dates_to_one_format(data):
    data['Travel Date'] = pd.to_datetime(data['Travel Date'], errors='coerce')
    data['Travel Date'] = data['Travel Date'].dt.strftime('%d-%m-%Y')
    return data

def preprocess_data(data):
    data = remove_useless_columns(data)
    data = convert_dates_to_one_format(data)
    data = hotel_details_encoding(data)
    data = split_columns_with_multiple_values(data)
    data = one_hot_encoding_on_columns(data)
    data = structuring_columns(data)
    return(data)

In [68]:
train = pd.read_csv('dataset\Train.csv')
validation = pd.read_csv('dataset\Test.csv')

final_train_data = preprocess_data(train)
final_validation_data = preprocess_data(validation)

In [69]:
### Displaying processed data

print('================TRAINING DATA==================')
display(final_train_data)
print(final_train_data.shape)

print('================VALIDATION DATA==================')
display(final_validation_data)
print(final_validation_data.shape)

================TRAINING DATA==================


,Itinerary,Places Covered,Travel Date,Hotel Details,Airline,Flight Stops,Meals,Sightseeing Places Covered,Per Person Price,type_Budget,type_Deluxe,type_Luxury,type_Premium,type_Standard,sc_Mumbai,sc_New Delhi,Not Available_ae,IndiGo_ae,Spicejet_ae,Go Air_ae,Air India_ae,Emirates_ae,Vistara_ae,Thai Airways_ae,AirAsia Indonesia_ae,Srilankan Airlines_ae,AirAsia_ae,Air India Express_ae,Malindo Air_ae,Malaysia Airlines_ae,Qantas Airways_ae,Air New Zealand_ae,Singapore Airlines_ae,Air Seychelles_ae,Etihad Airways_ae,AirAsia X_ae,Scoot_ae,Air Mauritius_ae,Oman Air_ae,Thai AirAsia_ae,Jetstar Asia_ae,Saudia_ae,Thai Vietjet Air_ae,EgyptAir_ae,Garuda Indonesia_ae,Silkair_ae,Bangkok Airways_ae,Kenya Airways_ae,Aeroflot_ae,Philippine Airlines_ae,Jetstar Airways_ae,Swiss_ae,Gulf Air_ae,Lufthansa_ae,Air Baltic_ae,Uzbekistan Airways_ae,Kuwait Airways_ae,China Southern_ae,Finnair_ae,Qatar Airways_ae,Airconnect_ae,China Eastern_ae,Norwegian_ae,British Airways_ae,Virgin Atlantic_ae,Turkish Airlines_ae,Brussels Airlines_ae,Cathay Pacific_ae,Air Austral_ae,South African Airways_ae,Ethiopian Airlines_ae,Sichuan Airlines_ae,New Delhi_pc,Shimla_pc,Manali_pc,Chandigarh_pc,Srinagar_pc,Pahalgam_pc,Udaipur_pc,Chittorgarh_pc,Munnar_pc,Kumarakom_pc,Allepey_pc,Kovalam and Poovar_pc,Pattaya_pc,Bangkok_pc,Cochin_pc,Jaipur_pc,Kasol_pc,Gulmarg_pc,Kanyakumari_pc,Guwahati_pc,Mount Abu_pc,Nainital_pc,Kausani_pc,Corbett_pc,Kodaikanal_pc,Dharamshala_pc,Ooty_pc,Mysore_pc,Agra_pc,Dubai_pc,Goa_pc,Gangtok_pc,Pelling_pc,Darjeeling_pc,Dalhousie_pc,Amritsar_pc,Coorg_pc,Bandipur_pc,Thekkady_pc,Lachung_pc,Bangalore_pc,Kandy_pc,Bentota_pc,Colombo_pc,Parwanoo_pc,Nameri_pc,Wayanad_pc,Kalimpong_pc,Bali_pc,Singapore_pc,Kovalam_pc,Negombo_pc,Nuwara Eliya_pc,Madurai_pc,Rameshwaram_pc,Leh_pc,Kullu_pc,Jodhpur_pc,Jaisalmer_pc,Port Blair_pc,Havelock_pc,Shillong_pc,Namchi_pc,Pondicherry_pc,Mahabalipuram_pc,Katra_pc,Mussoorie_pc,Abu Dhabi_pc,Rishikesh_pc,Kuala Lumpur_pc,Aurangabad_pc,Shirdi_pc,Kasauli_pc,Queenstown_pc,Kumbalgarh_pc,Ranikhet_pc,Niel Island_pc,Tirupati_pc,Kaziranga_pc,Ranthambhore_pc,Ajmer_pc,Mathura_pc,Haridwar_pc,Bikaner_pc,Auckland_pc,Rotorua_pc,Christchurch_pc,Sharjah_pc,Mauritius_pc,Praslin Island_pc,Mahe Island_pc,Poovar_pc,Varkala_pc,Pushkar_pc,Langkawi_pc,Yala_pc,Thimpu_pc,Paro_pc,Chail_pc,Tanjore_pc,Kumbakonam_pc,Tiruchirapally_pc,Patnitop_pc,Phuket_pc,Krabi town_pc,Siliguri_pc,Almora_pc,Punakha_pc,Pali_pc,Mumbai_pc,Alibagh_pc,Ganpatipule_pc,Lonavala And Khandala_pc,Sigiriya_pc,Galle_pc,Genting_pc,Bharatpur_pc,Cherrapunjee_pc,Palampur_pc,Ras Al Khaimah_pc,Penang_pc,Paris_pc,Bintan Island_pc,Nubra Valley_pc,Chintpurni_pc,Kangra_pc,Kabini_pc,Batam_pc,Lachen_pc,Dambulla_pc,Kuta_pc,Seminyak_pc,Nasik_pc,Saputara_pc,Jammu_pc,Ahmedabad_pc,Jamnagar_pc,Dwarka_pc,Somnath_pc,Anuradhapura_pc,Lucknow_pc,Varanasi_pc,Jakarta_pc,Johor Bahru_pc,Trivandrum_pc,Ranakpur_pc,Trincomalee_pc,Sonmarg_pc,Chennai_pc,Kanchipuram_pc,Nanded_pc,Ahmednagar_pc,Gokarna_pc,Melbourne_pc,Gold Coast_pc,Cairns_pc,Sydney_pc,Nawalgarh_pc,Bundi_pc,Prague_pc,Phuentsholing_pc,Moscow_pc,Saint Petersburg_pc,Hassan_pc,Hampi_pc,Franz Josef_pc,Kota Kinabalu_pc,Chikmangalur_pc,Kishangarh_pc,Mandawa_pc,Singapore Cruise_pc,Hong Kong_pc,Vadodara_pc,Joshimath_pc,Copenhagen_pc,Daman_pc,Helsinki_pc,Gwalior_pc,Orchha_pc,Khajuraho_pc,Alwar_pc,Oslo_pc,New Delhi_pc_hd,Shimla_pc_hd,Manali_pc_hd,Chandigarh_pc_hd,Srinagar_pc_hd,Pahalgam_pc_hd,Udaipur_pc_hd,Chittorgarh_pc_hd,Munnar_pc_hd,Kumarakom_pc_hd,Allepey_pc_hd,Kovalam and Poovar_pc_hd,Pattaya_pc_hd,Bangkok_pc_hd,Cochin_pc_hd,Jaipur_pc_hd,Kasol_pc_hd,Gulmarg_pc_hd,Kanyakumari_pc_hd,Guwahati_pc_hd,Mount Abu_pc_hd,Nainital_pc_hd,Kausani_pc_hd,Corbett_pc_hd,Kodaikanal_pc_hd,Dharamshala_pc_hd,Ooty_pc_hd,Mysore_pc_hd,Agra_pc_hd,Dubai_pc_hd,Goa_pc_hd,Gangtok_pc_hd,Pelling_pc_hd,Darjeeling_pc_hd,Dalhousie_pc_hd,Amritsar_pc_hd,Coorg_pc_hd,Bandipur_pc_hd,Thekkady_pc_hd,Lachung_pc_hd,Bangalore_pc_hd,Kandy_pc_hd,Bentota_pc_hd,Colombo_pc_hd,Parwanoo_pc_hd,Nameri_pc_hd,Wayanad_pc_hd,

(21000, 436)
================VALIDATION DATA==================


,Itinerary,Places Covered,Travel Date,Hotel Details,Airline,Flight Stops,Meals,Sightseeing Places Covered,type_Budget,type_Deluxe,type_Luxury,type_Premium,type_Standard,sc_Mumbai,sc_New Delhi,IndiGo_ae,Not Available_ae,Emirates_ae,Spicejet_ae,Air India_ae,Lufthansa_ae,Srilankan Airlines_ae,Vistara_ae,Go Air_ae,AirAsia X_ae,Malindo Air_ae,Bangkok Airways_ae,AirAsia Indonesia_ae,AirAsia_ae,Malaysia Airlines_ae,Jetstar Asia_ae,Oman Air_ae,Air India Express_ae,Singapore Airlines_ae,Silkair_ae,Thai Airways_ae,Thai AirAsia_ae,Cathay Pacific_ae,Qantas Airways_ae,Air Mauritius_ae,Scoot_ae,EgyptAir_ae,Air New Zealand_ae,Air Austral_ae,Saudia_ae,Aeroflot_ae,Air Seychelles_ae,Etihad Airways_ae,Swiss_ae,Air Baltic_ae,Turkish Airlines_ae,Thai Vietjet Air_ae,Qatar Airways_ae,Ethiopian Airlines_ae,Garuda Indonesia_ae,Goa_pc,Bandipur_pc,Munnar_pc,Cochin_pc,Thekkady_pc,Allepey_pc,Mauritius_pc,Kaziranga_pc,Guwahati_pc,Gangtok_pc,Lachen_pc,Lachung_pc,Paris_pc,Coorg_pc,Ooty_pc,Sharjah_pc,Kuala Lumpur_pc,Johor Bahru_pc,Amritsar_pc,Mysore_pc,Dharamshala_pc,Dalhousie_pc,Darjeeling_pc,Shimla_pc,Manali_pc,Chandigarh_pc,Bangalore_pc,New Delhi_pc,Agra_pc,Jaipur_pc,Kovalam and Poovar_pc,Bali_pc,Phuket_pc,Krabi town_pc,Pattaya_pc,Bangkok_pc,Colombo_pc,Kandy_pc,Madurai_pc,Kodaikanal_pc,Rameshwaram_pc,Kanyakumari_pc,Kovalam_pc,Udaipur_pc,Mount Abu_pc,Port Blair_pc,Havelock_pc,Jodhpur_pc,Abu Dhabi_pc,Dubai_pc,Singapore_pc,Jaisalmer_pc,Kabini_pc,Pelling_pc,Chittorgarh_pc,Shillong_pc,Haridwar_pc,Kasauli_pc,Srinagar_pc,Gulmarg_pc,Pahalgam_pc,Chail_pc,Nainital_pc,Kausani_pc,Ranikhet_pc,Corbett_pc,Kumarakom_pc,Tirupati_pc,Kalimpong_pc,Bentota_pc,Galle_pc,Bikaner_pc,Katra_pc,Leh_pc,Pondicherry_pc,Anuradhapura_pc,Sigiriya_pc,Nuwara Eliya_pc,Yala_pc,Hassan_pc,Hampi_pc,Nasik_pc,Saputara_pc,Cherrapunjee_pc,Mathura_pc,Thimpu_pc,Paro_pc,Kullu_pc,Bintan Island_pc,Negombo_pc,Pushkar_pc,Shirdi_pc,Trivandrum_pc,Ajmer_pc,Langkawi_pc,Mussoorie_pc,Parwanoo_pc,Varkala_pc,Genting_pc,Niel Island_pc,Chikmangalur_pc,Rishikesh_pc,Wayanad_pc,Ranthambhore_pc,Phuentsholing_pc,Punakha_pc,Kanchipuram_pc,Palampur_pc,Auckland_pc,Queenstown_pc,Patnitop_pc,Kuta_pc,Seminyak_pc,Mahabalipuram_pc,Mandawa_pc,Kumbakonam_pc,Tanjore_pc,Tiruchirapally_pc,Nubra Valley_pc,Aurangabad_pc,Ahmednagar_pc,Lonavala And Khandala_pc,Bharatpur_pc,Christchurch_pc,Trincomalee_pc,Kasol_pc,Batam_pc,Almora_pc,Poovar_pc,Namchi_pc,Gwalior_pc,Orchha_pc,Khajuraho_pc,Jammu_pc,Ranakpur_pc,Kumbalgarh_pc,Nanded_pc,Ahmedabad_pc,Sonmarg_pc,Kota Kinabalu_pc,Siliguri_pc,Kishangarh_pc,Chintpurni_pc,Kangra_pc,Praslin Island_pc,Mahe Island_pc,Chennai_pc,Hong Kong_pc,Mumbai_pc,Gokarna_pc,Lucknow_pc,Varanasi_pc,Prague_pc,Jamnagar_pc,Dwarka_pc,Somnath_pc,Nameri_pc,Jakarta_pc,Dambulla_pc,Ganpatipule_pc,Moscow_pc,Saint Petersburg_pc,Alwar_pc,Bundi_pc,Copenhagen_pc,Vadodara_pc,Pali_pc,Joshimath_pc,Nawalgarh_pc,Daman_pc,Ras Al Khaimah_pc,Franz Josef_pc,Rotorua_pc,Penang_pc,Melbourne_pc,Gold Coast_pc,Cairns_pc,Sydney_pc,30-01-2021_pc,Alibagh_pc,Goa_pc_hd,Bandipur_pc_hd,Munnar_pc_hd,Cochin_pc_hd,Thekkady_pc_hd,Allepey_pc_hd,Mauritius_pc_hd,Kaziranga_pc_hd,Guwahati_pc_hd,Gangtok_pc_hd,Lachen_pc_hd,Lachung_pc_hd,Paris_pc_hd,Coorg_pc_hd,Ooty_pc_hd,Sharjah_pc_hd,Kuala Lumpur_pc_hd,Johor Bahru_pc_hd,Amritsar_pc_hd,Mysore_pc_hd,Dharamshala_pc_hd,Dalhousie_pc_hd,Darjeeling_pc_hd,Shimla_pc_hd,Manali_pc_hd,Chandigarh_pc_hd,Bangalore_pc_hd,New Delhi_pc_hd,Agra_pc_hd,Jaipur_pc_hd,Kovalam and Poovar_pc_hd,Bali_pc_hd,Phuket_pc_hd,Krabi town_pc_hd,Pattaya_pc_hd,Bangkok_pc_hd,Colombo_pc_hd,Kandy_pc_hd,Madurai_pc_hd,Kodaikanal_pc_hd,Rameshwaram_pc_hd,Kanyakumari_pc_hd,Kovalam_pc_hd,Udaipur_pc_hd,Mount Abu_pc_hd,Port Blair_pc_hd,Havelock_pc_hd,Jodhpur_pc_hd,Abu Dhabi_pc_hd,Dubai_pc_hd,Singapore_pc_hd,Jaisalmer_pc_hd,Kabini_pc_hd,Pelling_pc_hd,Chittorgarh_pc_hd,Shillong_pc_hd,Haridwar_pc_hd,Kasauli_pc_hd,Srinagar_pc_hd,Gulmarg_pc_hd,Pahalgam_pc_hd,Chail_pc_hd,Nainital_pc_hd,Kausani_pc_hd,Ranikhet_pc_hd,Corbett_pc_hd,Kumarakom_pc_hd,Tirupati_pc_hd,Kalimpong_p

(9000, 415)


In [71]:
### Saving processed data to csv

final_train_data.to_csv('dataset\Train_prepared.csv') 
final_validation_data.to_csv('dataset\Test_prepared.csv')